In [2]:
from __future__ import annotations
import os, uuid
from typing import List
from PIL import Image
import chromadb
from sentence_transformers import SentenceTransformer
import numpy as np

In [6]:
# ---------------- Config ----------------
PERSIST_DIR = "./chroma_multimodal_local"
COLLECTION_NAME = "pets_local"

DOG_IMAGES = ["images/dog1.png", "images/dog2.png", "images/dog3.png"]
CAT_IMAGES = ["images/cat1.png", "images/cat2.png", "images/cat3.png"]

In [11]:
# ---------------- Helpers ----------------
def l2_normalize(vecs: np.ndarray) -> np.ndarray:
  norms = np.linalg.norm(vecs, axis=1, keepdims=True) + 1e-12
  return vecs / norms

def load_images(paths: List[str]) -> List[Image.Image]:
  """โหลดรูปจาก path ที่ให้มา"""
  return [Image.open(path).convert("RGB") for path in paths]

def encode_images(model: SentenceTransformer, images: List[Image.Image]) -> np.ndarray:
  """แปลงรูปเป็น embedding แล้ว normalize"""
  embs = model.encode(images, convert_to_numpy=True, batch_size=4, normalize_embeddings=False)
  return l2_normalize(embs)

def add_images_to_db(collection, images: List[str], labels: List[str], embs: np.ndarray):
    """เพิ่มรูปและ metadata เข้า Chroma"""
    doc_ids = [str(uuid.uuid4()) for _ in images]
    metadatas = [{"label": lbl, "filename": os.path.basename(path)} for lbl, path in zip(labels, images)]
    documents = [f"A {lbl} image ({os.path.basename(path)})" for lbl, path in zip(labels, images)]

    collection.add(
        ids=doc_ids,
        embeddings=embs.tolist(),
        metadatas=metadatas,
        documents=documents
    )
    print(f"Indexed {len(doc_ids)} images into collection '{collection.name}'.")


In [ ]:
# Step 1 - เตรียม path ของรูป
all_images = DOG_IMAGES + CAT_IMAGES
labels = ["dog"] * len(DOG_IMAGES) + ["cat"] * len(CAT_IMAGES)

# Step 2 - โหลด CLIP model
model = SentenceTransformer("clip-ViT-B-32")

# Step 3 - โหลดรูปจาก local
pil_images = load_images(all_images)

# Step 4 - embedding
img_embs = encode_images(model, pil_images)

/opt/miniconda3/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Step 5 - Setup Chroma
client = chromadb.Client(chromadb.config.Settings(persist_directory=PERSIST_DIR))
collection = client.get_or_create_collection(name=COLLECTION_NAME,  metadata={"hnsw:space": "cosine"})

In [ ]:
# Step 6 - เพิ่มรูปเข้า DB
add_images_to_db(collection, all_images, labels, img_embs)

Indexed 6 images into collection 'pets_local'.


In [ ]:
# Step 7 - ทดสอบ Query
query = "dog"
q_emb = model.encode([query], convert_to_numpy=True)
q_emb = l2_normalize(q_emb)

res = collection.query(
  query_embeddings=q_emb.tolist(),
  n_results=4,
  include=["metadatas", "documents", "distances"]
)

print(f"\n=== Query: {query} ===")
for rank, (meta, doc, dist) in enumerate(zip(res["metadatas"][0], res["documents"][0], res["distances"][0]), start=1):
  print(f"#{rank} -> {meta['filename']} ({meta['label']}) distance={dist:.4f}")
  print("desc:", doc)


=== Query: dog ===
#1 -> dog3.png (dog) distance=0.7287
desc: A dog image (dog3.png)
#2 -> dog2.png (dog) distance=0.7348
desc: A dog image (dog2.png)
#3 -> dog1.png (dog) distance=0.7422
desc: A dog image (dog1.png)
#4 -> cat1.png (cat) distance=0.7719
desc: A cat image (cat1.png)
